In [2]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [4]:
Institutions = "data/cc_institution_details.csv"
Institutions_db = pd.read_csv(Institutions,encoding="cp1252")

In [5]:
Institutions_db.head()

,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [6]:
len(Institutions_db)

3798

In [7]:
MF = Institutions_db.reset_index()

In [8]:
Inst = MF[["unitid","chronname","city","state","level","control","student_count","med_sat_value","aid_value","grad_100_value","grad_150_value"]]

In [9]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,322,NaN,2540.0,0.0,66.7
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
...,...,...,...,...,...,...,...,...,...,...,...
3793,461528,Grace College of Divinity,Fayetteville,North Carolina,4-year,Private not-for-profit,110,NaN,4860.0,0.0,0.0
3794,462354,John Paul the Great Catholic University,Escondido,California,4-year,Private not-for-profit,140,1069.0,9395.0,51.5,51.5
3795,466921,Chamberlain College of Nursing-Missouri,St. Louis,Missouri,4-year,Private for-profit,488,NaN,3291.0,37.5,37.5
3796,474881,Minneapolis Media Institute,Edina,Minnesota,2-year,Private for-profit,147,NaN,3434.0,42.1,55.9


In [10]:
Inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unitid          3798 non-null   int64  
 1   chronname       3798 non-null   object 
 2   city            3798 non-null   object 
 3   state           3798 non-null   object 
 4   level           3798 non-null   object 
 5   control         3798 non-null   object 
 6   student_count   3798 non-null   int64  
 7   med_sat_value   1337 non-null   float64
 8   aid_value       3797 non-null   float64
 9   grad_100_value  3467 non-null   float64
 10  grad_150_value  3467 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 326.5+ KB


In [11]:
Inst = Inst.dropna()

In [12]:
Inst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1319 entries, 0 to 3794
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unitid          1319 non-null   int64  
 1   chronname       1319 non-null   object 
 2   city            1319 non-null   object 
 3   state           1319 non-null   object 
 4   level           1319 non-null   object 
 5   control         1319 non-null   object 
 6   student_count   1319 non-null   int64  
 7   med_sat_value   1319 non-null   float64
 8   aid_value       1319 non-null   float64
 9   grad_100_value  1319 non-null   float64
 10  grad_150_value  1319 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 123.7+ KB


In [13]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
5,100751,University of Alabama at Tuscaloosa,Tuscaloosa,Alabama,4-year,Public,29440,1171.0,10390.0,42.7,66.7
...,...,...,...,...,...,...,...,...,...,...,...
3572,446048,Ave Maria University,Ave Maria,Florida,4-year,Private not-for-profit,906,1070.0,15463.0,54.7,57.7
3639,448840,University of South Florida at St. Petersburg,St. Petersburg,Florida,4-year,Public,4025,1070.0,5005.0,22.0,42.7
3651,449764,Visible School,Memphis,Tennessee,4-year,Private not-for-profit,118,1070.0,4958.0,0.0,0.0
3717,454184,King's College (N.Y.),New York,New York,4-year,Private not-for-profit,516,1160.0,19698.0,56.8,61.4


In [14]:
 Inst.groupby(["level"]).count()

,unitid,chronname,city,state,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
level,,,,,,,,,,
2-year,22,22,22,22,22,22,22,22,22,22
4-year,1297,1297,1297,1297,1297,1297,1297,1297,1297,1297


In [15]:
len(Institutions_db)

3798

In [16]:
Alabama = Inst.loc[(Inst["state"] == "Alabama")]

In [17]:
Alabama

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
5,100751,University of Alabama at Tuscaloosa,Tuscaloosa,Alabama,4-year,Public,29440,1171.0,10390.0,42.7,66.7
7,100830,Auburn University at Montgomery,Montgomery,Alabama,4-year,Public,4322,970.0,4327.0,9.9,27.1
8,100858,Auburn University,Auburn University,Alabama,4-year,Public,19799,1215.0,8875.0,37.6,67.9
9,100937,Birmingham-Southern College,Birmingham,Alabama,4-year,Private not-for-profit,1188,1177.0,20385.0,53.2,61.9
21,101435,Huntingdon College,Montgomery,Alabama,4-year,Private not-for-profit,1110,999.0,11447.0,30.4,43.2
23,101480,Jacksonville State University,Jacksonville,Alabama,4-year,Public,7588,1036.0,6289.0,10.1,30.8


In [18]:
Howmanystates = Inst.groupby(["state"]).count()

In [19]:
Howmanystates

,unitid,chronname,city,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
state,,,,,,,,,,
Alabama,23,23,23,23,23,23,23,23,23,23
Alaska,1,1,1,1,1,1,1,1,1,1
Arizona,6,6,6,6,6,6,6,6,6,6
Arkansas,18,18,18,18,18,18,18,18,18,18
California,81,81,81,81,81,81,81,81,81,81
Colorado,15,15,15,15,15,15,15,15,15,15
Connecticut,14,14,14,14,14,14,14,14,14,14
Delaware,3,3,3,3,3,3,3,3,3,3
District of Columbia,6,6,6,6,6,6,6,6,6,6


In [35]:
Inst.to_csv("Inst.csv")

In [22]:
len(Howmanystates)

51

In [23]:
InstSort = Inst.sort_values("control")

In [24]:
InstSort

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
1758,192271,LIM College,New York,New York,4-year,Private for-profit,1513,907.0,6043.0,40.9,45.6
1567,182661,Daniel Webster College,Nashua,New Hampshire,4-year,Private for-profit,632,962.0,10377.0,33.5,46.4
1874,197151,School of Visual Arts,New York,New York,4-year,Private for-profit,3636,1020.0,10995.0,55.3,64.6
2466,217749,Bob Jones University,Greenville,South Carolina,4-year,Private for-profit,2913,1070.0,8359.0,51.4,61.4
464,130183,Post University,Waterbury,Connecticut,4-year,Private for-profit,7371,865.0,7187.0,26.1,34.8
...,...,...,...,...,...,...,...,...,...,...,...
1610,185129,New Jersey City University,Jersey City,New Jersey,4-year,Public,6437,916.0,7258.0,4.9,31.7
1611,185262,Kean University,Union,New Jersey,4-year,Public,12078,905.0,6286.0,18.9,46.2
1615,185590,Montclair State University,Montclair,New Jersey,4-year,Public,15431,970.0,6094.0,34.4,63.4
1544,181464,University of Nebraska at Lincoln,Lincoln,Nebraska,4-year,Public,19376,1147.0,6699.0,32.3,66.8


In [25]:
Grads = "data/cc_institution_grads.csv"
Grads_db = pd.read_csv(Grads)

In [33]:
Grads_db

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
1,100760,2011,M,X,2y all,185.0,NaN,40.0,NaN,21.6
2,100760,2011,F,X,2y all,261.0,NaN,65.0,NaN,24.9
3,100760,2011,B,W,2y all,348.0,NaN,86.0,NaN,24.7
4,100760,2011,M,W,2y all,162.0,NaN,35.0,NaN,21.6
...,...,...,...,...,...,...,...,...,...,...
1302097,168591,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302098,168740,2002,F,Ai,4y other,0.0,NaN,0.0,NaN,NaN
1302099,169716,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302100,170082,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN


In [52]:
merged_df = pd.merge(Inst, Grads_db, how='outer', on='unitid')
merged_df.groupby("gender")
merged_df.head()

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051.0,823.0,7142.0,10.0,29.1,2011,B,X,4y other,NaN,NaN,NaN,NaN,NaN
1,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051.0,823.0,7142.0,10.0,29.1,2011,M,X,4y other,NaN,NaN,NaN,NaN,NaN
2,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051.0,823.0,7142.0,10.0,29.1,2011,F,X,4y other,NaN,NaN,NaN,NaN,NaN
3,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051.0,823.0,7142.0,10.0,29.1,2011,B,W,4y other,NaN,NaN,NaN,NaN,NaN
4,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051.0,823.0,7142.0,10.0,29.1,2011,M,W,4y other,NaN,NaN,NaN,NaN,NaN
